In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email='your@email.com'
#search Genbank, returns accession numbers (up to 100)
handle=Entrez.esearch(db='nucleotide', retmax=3, term="rbcl[All Fields]", idtype="acc") 
record = Entrez.read(handle)
print(handle)

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import squarify

df= pd.read_csv("../webmd.csv")
#"C:\Users\52812\"
df['Date'] = pd.to_datetime(df['Date'], format = '%m/%d/%Y')
df.dropna(subset=df.columns.difference(['Reviews']), how='all')

#Sex
df= df[df.Sex != ' ']
df= df[df.Age != ' ']

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331577 entries, 0 to 362805
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Age            331577 non-null  object        
 1   Condition      331577 non-null  object        
 2   Date           331577 non-null  datetime64[ns]
 3   Drug           331577 non-null  object        
 4   DrugId         331577 non-null  int64         
 5   EaseofUse      331577 non-null  int64         
 6   Effectiveness  331577 non-null  int64         
 7   Reviews        331545 non-null  object        
 8   Satisfaction   331577 non-null  int64         
 9   Sex            331577 non-null  object        
 10  Sides          331577 non-null  object        
 11  UsefulCount    331577 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 32.9+ MB
None


In [ ]:
#Age
#print(df.Age.unique())
#df= df[df.Age != ' ']
#print(df.Age.value_counts())

df_age= df.groupby(['Age']).count()['DrugId'].reset_index()

squarify.plot(sizes=df_age['DrugId'], label=df_age['Age'], alpha= 1, pad = 0.2, color = sns.color_palette("pastel"))

plt.text(50, 100, 'Age distribution of WebMD', fontsize = 22, color='grey', horizontalalignment='center',
         verticalalignment='bottom', zorder = 20) 
plt.axis('off')
plt.show()

In [ ]:
#Most common conditions by age groups
df_cond= df.groupby(['Age', 'Condition']).count()['DrugId'].reset_index()
df_cond= df_cond[df_cond.Condition != 'Other']

Ages = df.Age.unique()

df_cond_top= []
m=  {}

for i in Ages:
    m = df_cond[df_cond.Age == i]
    m = m.sort_values(by='DrugId', ascending= False).head()#.to_string(header=False)
    df_cond_top.append(m)

df_cond_top = pd.concat(df_cond_top)
print(df_cond_top)

In [ ]:
df.head()
#.drop(df.index[0])
print(df_cond_top)

In [ ]:
#Bar for unique values
#EaseofUse
print(df.EaseofUse.value_counts())
#Condition
print(df.Condition.value_counts())
#Satisfaction
print(df.Satisfaction.value_counts())
#Drug
print(df.Drug.value_counts())
#Effectiveness
print(df.Effectiveness.value_counts())

In [ ]:
df_cond= df.groupby(['Age', 'Condition', 'Effectiveness', 'Sex']).mean()['DrugId'].reset_index()

sns.set_theme(style="ticks", palette="pastel")

sns.catplot(
    data=df_cond , kind="bar",
    x="Effectiveness", y = 'DrugId', hue="Sex", aspect = 2,
    #ci="sd", 
    alpha=.6, height=6, palette = 'pastel'
)

In [ ]:
df_sat= df.groupby(['Date', 'Sex']).mean()['Effectiveness'].reset_index()
ax = sns.set(rc={'figure.figsize':(15,10)})

plt.title('Average Price of Avocado by its type 2015-2018')
plt.xticks(rotation =45)
ax = sns.lineplot(x="Date",y="Effectiveness", hue='Sex', data=df_sat)

In [ ]:
x = df.Drug.value_counts()
x

In [ ]:
#Most common conditions
df_drug= df.groupby(['Condition']).count()['DrugId'].reset_index()
df_drug= df_drug[df_drug.Condition != 'Other']
df_drug= df_drug[df_drug.DrugId > 100]
df_drug = df_drug.sort_values(by='DrugId', ascending= False)
#df_drug = df_drug.head(50)
print(df_drug.head(50))

In [ ]:
df.Condition.value_counts()